In [1]:
from pyleotups import Dataset
import pandas as pd

# Will need to import the pylipd packages

In [2]:
ds=Dataset()

In [3]:
ds.search_studies(noaa_id=27490)

Parsing NOAA studies: 100%|████████████████████████████████████████████| 1/1 [00:00<00:00, 2283.24it/s]


,StudyID,XMLID,StudyName,DataType,EarliestYearBP,MostRecentYearBP,EarliestYearCE,MostRecentYearCE,StudyNotes,ScienceKeywords,Investigators,Publications,Sites,Funding
0,27490,67635,Palmyra 860 Year Modern and Fossil Coral Oxyge...,CORALS AND SCLEROSPONGES,804,-56,1146,2006,Monthly-resolved coral aragonite oxygen isotop...,None,"Sylvia Dee, Kim Cobb, Julien Emile-Geay, Toby ...","[{'Author': 'Sylvia G. Dee, Kim M. Cobb, Julie...","[[{'DataTableID': '41697', 'DataTableName': 'P...",[{'fundingAgency': 'US National Science Founda...


In [4]:
ds.get_summary()

,StudyID,XMLID,StudyName,DataType,EarliestYearBP,MostRecentYearBP,EarliestYearCE,MostRecentYearCE,StudyNotes,ScienceKeywords,Investigators,Publications,Sites,Funding
0,27490,67635,Palmyra 860 Year Modern and Fossil Coral Oxyge...,CORALS AND SCLEROSPONGES,804,-56,1146,2006,Monthly-resolved coral aragonite oxygen isotop...,None,"Sylvia Dee, Kim Cobb, Julien Emile-Geay, Toby ...","[{'Author': 'Sylvia G. Dee, Kim M. Cobb, Julie...","[[{'DataTableID': '41697', 'DataTableName': 'P...",[{'fundingAgency': 'US National Science Founda...


In [7]:
bib, df_pub = ds.get_publications()
df_pub.head()

,Author,Title,Journal,Year,Volume,Number,Pages,Type,DOI,URL,CitationKey,StudyID,StudyName
0,"Sylvia G. Dee, Kim M. Cobb, Julien Emile-Geay,...",No consistent ENSO response to volcanic forcin...,Science,2020,367,6485,1477-1481,publication,10.1126/science.aax2000,http://dx.doi.org/10.1126/science.aax2000,Charles_Consistent_2020_27490,27490,Palmyra 860 Year Modern and Fossil Coral Oxyge...


In [9]:
df_tables = ds.get_tables()
df_tables

,DataTableID,DataTableName,TimeUnit,FileURL,Variables,FileDescription,TotalFilesAvailable,SiteID,SiteName,LocationName,Latitude,Longitude,MinElevation,MaxElevation,StudyID,StudyName
0,41697,Palmyra2020d18O,CE,https://www.ncei.noaa.gov/pub/data/paleo/coral...,"[age, d18O]",NOAA Template File,1,1900,Palmyra Island,Ocean>Pacific Ocean>Central Pacific Ocean,5.8664,-162.12,-9,-9,27490,Palmyra 860 Year Modern and Fossil Coral Oxyge...


In [10]:
df_geo = ds.get_geo()
df_geo

,StudyID,DataType,SiteID,SiteName,LocationName,Latitude,Longitude,MinElevation,MaxElevation
0,27490,CORALS AND SCLEROSPONGES,1900,Palmyra Island,Ocean>Pacific Ocean>Central Pacific Ocean,5.8664,-162.12,-9,-9


In [11]:
dfs = ds.get_data(dataTableIDs="41697")

In [12]:
dfs[0].head()

,age,d18O
0,1146.375,-4.749
1,1146.4583,-4.672
2,1146.5417,-4.724
3,1146.625,-4.717
4,1146.7083,-4.8947


In [13]:
dfs[0].attrs

{'variables': ['age', 'd18O'],
 'NOAAStudyId': '27490',
 'StudyName': 'Palmyra 860 Year Modern and Fossil Coral Oxygen Isotope Data'}